In this notebook, I will be pulling imagery from the year before a fire in the Great Smoky Mountains and after the fire. We will be comparing NDVI, NBR, and MSMI and pulling species occurrence records from before and after the fire and see how it overlaps with occurrence records.

#### Declare ROI

#### Pull NEON Imagery from GEE

#### Pull Records from GBIF